In this lab we turn a pre-trained BERT model into a trainable Keras layer and apply it to the Aspect-Based Sentiment Analysis (ABSA). You can find the task description from (https://aclanthology.org/D19-1654.pdf).
This task provides a review text dataset with aspects.
Given a review and an aspect, we need to classify the sentiment conveyed towards this aspect on a three-point scale: POSITIVE, NEUTRAL or NEGATIVE.
This is a multi-class classification task.

BERT (Bidirectional Embedding Representations from Transformers) is a model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. We demonstrate how to integrate BERT as a custom Keras layer using the Huggingface library.

In this lab, you will learn:

1) How to use the Huggingface library.

2) How to integrate BERT in the models built previously.

3) How to use the TPU from Colab. **Note**: Running BERT on the CPU will be very slow. Thus we recommend you to do this lab on a Colab TPU provided by Google.

# Lab 3 - Aspect-Based Sentiment Analysis with BERT

In [1]:
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt

from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers import Dense
from keras.models import Model


2024-02-29 13:48:08.023591: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 13:48:08.064623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 13:48:08.064666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 13:48:08.065816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 13:48:08.072229: I tensorflow/core/platform/cpu_feature_guar

We first need to install the Huggingface's Transformers package. You can find the relevant doc from [here](https://huggingface.co/transformers/index.html).

In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


## Preprocessing and Tokenization

In this lab we will use DistilBERT (https://arxiv.org/pdf/1910.01108.pdf) instead of BERT: DistilBERT is a smaller and faster Transformer model trained via distilling BERT. It has 40% less parameters than the bert-base-uncased model. It runs 60% faster, while preserving 97% of BERT’s performance as measured on the GLUE language understanding benchmark.

It is easy to switch between DistilBERT and BERT using the Huggingface transformers package. This package provides many pre-trained and pre-built models that are easy to use via a few lines of code.

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for this particular model.
We can get the DistilBERT tokenizer from the **DistilBertTokenizer.from_pretrained** method.

In [3]:

from transformers import DistilBertTokenizer, RobertaTokenizer
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick a pre-trained model

# Defining DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length,
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')

    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length,
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])

    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Then we can use this tokenizer to tokenize our data. When working with word2vec and GloVe, you tokenized sentences into words yourself and then converted the tokens into GloVe indices. But in BERT, we must use the BERT tokenizer that uses sub-word tokens.

For example, for the sentence: **This is a pretrained model.** our previous word-based tokenizer will generate the following tokens:

**"this", "is", "a", "pretrained", "model", "."**

Then you will find out that the token "pretrained" is not in the GloVe word dictionary. Thus we can not assign it a trained word vector.

The BERT tokenizer will separate the word "pretrained" into three sub-word tokens:

**'pre', '##train', '##ed'**

BERT thus uses these three token vectors to represent the word "pretrained". You will also see that the BERT tokenizer adds the special [CLS] token and the sentence separator [SEP] token.

## Loading Data

### Downloading and preprocessing data

Unlike the IMDB dataset that is included and preprocessed by Keras, the dataset we will be using is the aspect-based sentiment analysis (ABSA)  dataset, which consists of 5,297 labeled reviews. These are split into 4,297 reviews for training and 500 reviews for testing and validation, respectively.

For ABSA, sentiment polarities were assigned with respect to the aspect terms.  The start and end positions for each aspect term are provided.


In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/train.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/val.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/test.xml')


# The code is modified from https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data_process/utils.py
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = [text , term,  polarity , start , end]
            data.append(piece)
    return data
train = parse_sentence_term("train.xml",True)
dev = parse_sentence_term("val.xml",True)
test = parse_sentence_term("test.xml",True)

print("Training entries: {}".format(len(train)))
print("Test entries: {}".format(len(test)))


Training entries: 11186
Test entries: 1336


Let’s first see some examples from the data:

In [6]:
print("SENTENCE \t ASPECT \t LABEL \t ASPECT-START-INDEX \t ASPECT-END-INDEX")
print(train[0])
print(train[1])
print(train[2])
print(train[3])
print(train[4])

SENTENCE 	 ASPECT 	 LABEL 	 ASPECT-START-INDEX 	 ASPECT-END-INDEX
['the decor is not special at all but their food and amazing prices make up for it.', 'decor', 'negative', '4', '9']
['the decor is not special at all but their food and amazing prices make up for it.', 'food', 'positive', '42', '46']
['the decor is not special at all but their food and amazing prices make up for it.', 'prices', 'positive', '59', '65']
['when tables opened up, the manager sat another party before us.', 'tables', 'neutral', '5', '11']
['when tables opened up, the manager sat another party before us.', 'manager', 'negative', '27', '34']


In [7]:
# Please write your code to generate the following data
# x_train_review_int
# x_train_review_masks
# x_train_aspect_int
# x_train_aspect_masks

# x_dev_review_int
# x_dev_review_masks
# x_dev_aspect_int
# x_dev_aspect_masks

# x_test_review_int
# x_test_review_masks
# x_test_aspect_int
# x_test_aspect_masks

# your code goes here
train_reviews=[item[0] for item in train]
train_aspects=[item[1] for item in train]
dev_reviews=[item[0] for item in dev]
dev_aspects=[item[1] for item in dev]
test_reviews=[item[0] for item in test]
test_aspects=[item[1] for item in test]

# Tokenization for Training Dataset
x_train_review_int, x_train_review_masks, _ = tokenize(train_reviews, tokenizer)
x_train_aspect_int, x_train_aspect_masks, _ = tokenize(train_aspects, tokenizer)

# Tokenization for Development (Validation) Dataset
x_dev_review_int, x_dev_review_masks, _ = tokenize(dev_reviews, tokenizer)
x_dev_aspect_int, x_dev_aspect_masks, _ = tokenize(dev_aspects, tokenizer)

# Tokenization for Testing Dataset
x_test_review_int, x_test_review_masks, _ = tokenize(test_reviews, tokenizer)
x_test_aspect_int, x_test_aspect_masks, _ = tokenize(test_aspects, tokenizer)


# You can check the results as follows:
assert len(x_train_aspect_int) == len(train)
assert len(x_train_aspect_masks) == len(x_train_aspect_int)
assert len(x_test_aspect_int) == len(test)
assert len(x_test_aspect_masks) == len(x_test_aspect_int)
print("x_dev_aspect_int[0]:")
print(x_dev_aspect_int[0])
print("x_dev_aspect_masks[0]:")
print(x_dev_aspect_masks[0])
print("x_dev_review_int[0]:")
print(x_dev_review_int[0])
print("x_dev_review_masks[0]:")
print(x_dev_review_masks[0])

x_dev_aspect_int[0]:
[ 101 8974  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_aspect_masks[0]:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_review_int[0]:
[  101  2044  1037  323

Using the BERT `tokenize` function above, we can turn text and aspect words to integers:


We one-hot encode the labels, using 4 (Binary:100) to represent "positive", 2 (Binary:010) for "neutral", and 1 (Binary:001) for "negative".

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append([0,0,1])
    elif example[2].lower() == "neutral":
      y.append([0,1,0])
    else:
      # assert example[2].lower() == "positive"
      y.append([1,0,0])
  return y

y_train = label2int(train)
y_dev = label2int(dev)
y_test = label2int(test)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


Now we have almost finished the data preprocessing. Unlike in the previous labs, there are two model inputs: review and aspect. The easiest way is to combine the review and aspect into one sentence and then input it into the model.

In [9]:
# Please write your code to combine sentences and aspect words into the following varibles

# x_train_int
# x_train_masks
# x_dev_int
# x_dev_masks
# x_test_int
# x_test_masks

# Tips:
# 1) Use the special token [SEP] to concatenate sentences and aspect words
# 2) Make sure they are paded/truncated to a max length

# your code goes here
def combineSentAndAspect(sentences, aspect_words, tokenizer, max_length=128):
    combined_texts = [sentence + " [SEP] " + aspect for sentence, aspect in zip(sentences, aspect_words)]
    input_ids, attention_masks = [], []

    for text in combined_texts:
        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    return np.array(input_ids), np.array(attention_masks)


x_train_int, x_train_masks = combineSentAndAspect(train_reviews, train_aspects, tokenizer)
x_dev_int, x_dev_masks = combineSentAndAspect(dev_reviews, dev_aspects, tokenizer)
x_test_int, x_test_masks = combineSentAndAspect(test_reviews, test_aspects, tokenizer)

# Don't forget the to use the np.array function to wrap the outputs
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0]) # sentence + aspect


[  101  2044  1037  3232  1997  8974  1010  1996 18726  1011  1011  1045
  2066  1996 27940  1013 24792  2621  4897  1998  1996 13675 11514  6508
 26852  1011  1011  2175  2091  2307  1012   102  8974   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## Model 1: Prebuilt Sequence Classification



The Huggingface transformer package provides many prebuilt models. Now let us try a sequence classification model based on DistilBERT.

The models with BERT are much bigger than our previous models. To run it faster, we can use TPU. Detailed guidelines on how to use TPU can be found from https://www.tensorflow.org/guide/tpu.

In [10]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=3)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
  transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
  input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  X = transformer_model(input_ids, input_masks_ids)
  return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)


use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model on TPU:
  with strategy.scope():
    model = create_TFDistilBertForSequenceClassification()
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model = create_TFDistilBertForSequenceClassification()
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


2024-02-29 13:48:30.877527: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model tha

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_token (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 masked_token (InputLayer)   [(None, 128)]                0         []                            
                                                                                                  
 tf_distil_bert_for_sequenc  TFSequenceClassifierOutput   6695577   ['input_token[0][0]',         
 e_classification (TFDistil  (loss=None, logits=(None,    9          'masked_token[0][0]']        
 BertForSequenceClassificat  3),                                                                  
 ion)                         hidden_states=None, atten                                       

In [13]:
history = model.fit([x_train_int_np[:50],x_train_masks_np[:50]],
                    y_train[:50],
                    epochs=3,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/3
1/1 [==============================] - 174s 174s/step - loss: 3.3769 - accuracy: 0.3600 - val_loss: 4.0689 - val_accuracy: 0.4535
Epoch 2/3
1/1 [==============================] - 148s 148s/step - loss: 3.6546 - accuracy: 0.4800 - val_loss: 5.0714 - val_accuracy: 0.3026
Epoch 3/3
1/1 [==============================] - 151s 151s/step - loss: 4.4728 - accuracy: 0.3200 - val_loss: 6.7807 - val_accuracy: 0.3026


In [14]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 168s 4s/step - loss: 6.7893 - accuracy: 0.2994
[6.789268493652344, 0.299401193857193]



## Model 2: Neural bag of words using BERT

Now we want to build the neural network model. We  are going to have a hidden layer with 16 hidden units.

First, we want to transform each index to a BERT embedded vector and then average all vectors to a single one. It has been showed that unweighted average of word vectors outperforms many complicated networks that model semantic and syntactic compositionality. As an example you can take a look at this: (http://anthology.aclweb.org/P/P15/P15-1162.pdf)

To average we need to ignore padded zeros:

In [15]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [16]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [17]:
import tensorflow as tf
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  # your code goes here
  input_ids_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_ids')
  input_masks_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

  pooled_sent= GlobalAveragePooling1DMasked()(embedded_sent)
  # Hint: you can use the following parameters: activation='sigmoid',kernel_initializer='glorot_uniform'
  hidden_output= Dense(hdepth, activation='sigmoid', kernel_initializer='glorot_uniform')(pooled_sent)
  # Hint: you can use the following parameters: activation='softmax',kernel_initializer='glorot_uniform'
  label= Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(hidden_output)

  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [18]:
model2.summary()

Model: "Model2_BERT"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 input_masks (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 tf_distil_bert_model (TFDi  TFBaseModelOutput(last_hid   6636288   ['input_ids[0][0]',           
 stilBertModel)              den_state=(None, 128, 768)   0          'input_masks[0][0]']         
                             , hidden_states=None, atte                                           
                             ntions=None)                                               

In [19]:
history = model2.fit([x_train_int_np[:50],x_train_masks_np[:50]],
                    y_train[:50],
                    epochs=3,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/3
1/1 [==============================] - 163s 163s/step - loss: 0.9343 - accuracy: 0.2000 - val_loss: 0.6368 - val_accuracy: 0.4535
Epoch 2/3
1/1 [==============================] - 157s 157s/step - loss: 0.6360 - accuracy: 0.4800 - val_loss: 0.6460 - val_accuracy: 0.4535
Epoch 3/3
1/1 [==============================] - 155s 155s/step - loss: 0.6332 - accuracy: 0.4800 - val_loss: 0.6331 - val_accuracy: 0.4535


In [20]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 168s 4s/step - loss: 0.6328 - accuracy: 0.4543
[0.6327853798866272, 0.454341322183609]


## Model 3: CNN or LSTM with BERT

Please follow the architecture for model2 to construct a CNN or an LSTM model on the top of BERT.

In [22]:
# your code goes here

def create_lstm_on_bert():
    input_ids_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_ids')
    input_masks_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')

    bert_embeddings = get_BERT_layer()
    embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

    lstm_layer = LSTM(units=100)(embedded_sent)

    label = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(lstm_layer)

    return Model(inputs=[input_ids_in, input_masks_in], outputs=label, name='Model3_LSTM')


use_tpu = True
if use_tpu:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    with strategy.scope():
      model3 = create_lstm_on_bert()
      optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
      model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

else:
    model3 = create_lstm_on_bert()
    model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [23]:
model3.summary()

Model: "Model3_LSTM"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 input_masks (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 tf_distil_bert_model_1 (TF  TFBaseModelOutput(last_hid   6636288   ['input_ids[0][0]',           
 DistilBertModel)            den_state=(None, 128, 768)   0          'input_masks[0][0]']         
                             , hidden_states=None, atte                                           
                             ntions=None)                                               

In [24]:
history = model3.fit([x_train_int_np[:50],x_train_masks_np[:50]],
                    y_train[:50],
                    epochs=3,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/3
1/1 [==============================] - 173s 173s/step - loss: 0.7397 - accuracy: 0.2200 - val_loss: 0.7445 - val_accuracy: 0.4535
Epoch 2/3
1/1 [==============================] - 160s 160s/step - loss: 0.7050 - accuracy: 0.4800 - val_loss: 0.6835 - val_accuracy: 0.4535
Epoch 3/3
1/1 [==============================] - 166s 166s/step - loss: 0.6525 - accuracy: 0.4800 - val_loss: 0.7240 - val_accuracy: 0.3026


In [ ]:
results = model3.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

41/42 [============================>.] - ETA: 4s - loss: 0.7219 - accuracy: 0.3011